#

# COGS 118C Final Project - Jupyter Notebook analysis
## Analyzing differing frequencies bands’ contributions to wake v. sleep states in rats

Calixto Calangi

Valerie Ortiz 

Indrani Vairagare   

In [ ]:
### code imported for conducting analysis ###
